In [30]:
import numpy as np
from glob import glob
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

In [31]:
# Get images' names
non_demented = glob('./data/Non_Demented/*.jpg')
mild_demented = glob('./data/Mild_Demented/*.jpg')
moderate_demented = glob('./data/Moderate_Demented/*.jpg')
very_mild_demented = glob('./data/Very_Mild_Demented/*.jpg')

In [33]:
def view_image(filename):
    img = mpimg.imread(filename)
    plt.imshow(img)
    print(f'Image shape: {img.shape}')


In [ ]:
print(f'Total amount of non_demented: {len(non_demented)}')
view_image(no)